In [42]:
import twelvedata


In [44]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)
from key import twelveDataKey as api_key

Mounted at /content/drive


In [47]:
import time as t
from twelvedata import TDClient
import pandas as pd
from datetime import datetime, timedelta, time
import os
from google.colab import drive

# Initialize the client
td = TDClient(apikey=api_key)

# Mount Google Drive
drive.mount('/content/drive')

def fetch_stock_data(symbol):
    # Define the date range
    start_date = datetime(2021, 1, 1)
    end_date = datetime(2024, 12, 31)

    # Define market hours
    market_open_time = time(9, 30)  # 9:30 AM
    market_close_time = time(16, 0)  # 4:00 PM

    all_data = []

    # Iterate through each month in the date range
    current_date = start_date
    while current_date <= end_date:
        # Calculate the last day of the current month
        if current_date.month == 12:
            next_month = current_date.replace(year=current_date.year + 1, month=1, day=1)
        else:
            next_month = current_date.replace(month=current_date.month + 1, day=1)
        last_day_of_month = next_month - timedelta(days=1)

        # Define the start and end times for the entire month
        month_start = datetime.combine(current_date, market_open_time)
        month_end = datetime.combine(last_day_of_month, market_close_time)

        # Fetch data for the entire month
        try:
            print(f"Fetching {symbol} data for {current_date.strftime('%Y-%m')}...")
            data = td.time_series(
                symbol=symbol,
                interval="30min",
                start_date=month_start.strftime("%Y-%m-%d %H:%M:%S"),
                end_date=month_end.strftime("%Y-%m-%d %H:%M:%S"),
                outputsize=5000
            ).as_pandas()

            if not data.empty:
                all_data.append(data)
                print(f"Successfully fetched {len(data)} rows for {symbol} in {current_date.strftime('%Y-%m')}.")
            else:
                print(f"No data returned for {symbol} in {current_date.strftime('%Y-%m')} (market may have been closed).")

        except Exception as e:
            print(f"Error fetching {symbol} data for {current_date.strftime('%Y-%m')}: {e}")

        # Move to the next month
        current_date = next_month

        # Add delay between requests to avoid hitting API limits
        t.sleep(10)  # 10-second delay (6 requests per minute)

    # Combine all chunks into a single DataFrame
    if all_data:
        full_data = pd.concat(all_data)
        # Remove duplicates (if any)
        full_data = full_data[~full_data.index.duplicated(keep='first')]
        # Save to Google Drive
        full_data.to_csv(f'/content/drive/My Drive/{symbol}_data.csv', index=True)
        print(f"{symbol} data saved to Google Drive as {symbol}_data.csv")
    else:
        print(f"No data collected for {symbol}")

# Pull data for AAPL and MSFT
fetch_stock_data("AAPL")
fetch_stock_data("MSFT")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Fetching AAPL data for 2021-01...
Successfully fetched 247 rows for AAPL in 2021-01.
Fetching AAPL data for 2021-02...
Successfully fetched 247 rows for AAPL in 2021-02.
Fetching AAPL data for 2021-03...
Successfully fetched 299 rows for AAPL in 2021-03.
Fetching AAPL data for 2021-04...
Successfully fetched 273 rows for AAPL in 2021-04.
Fetching AAPL data for 2021-05...
Successfully fetched 260 rows for AAPL in 2021-05.
Fetching AAPL data for 2021-06...
Successfully fetched 286 rows for AAPL in 2021-06.
Fetching AAPL data for 2021-07...
Successfully fetched 273 rows for AAPL in 2021-07.
Fetching AAPL data for 2021-08...
Successfully fetched 286 rows for AAPL in 2021-08.
Fetching AAPL data for 2021-09...
Successfully fetched 273 rows for AAPL in 2021-09.
Fetching AAPL data for 2021-10...
Successfully fetched 273 rows for AAPL in 2021-10.
Fetching AAPL data fo